In [1]:
%load_ext autoreload
%autoreload 2


In [10]:
import os

os.environ["HCONFIG_NAME"] = "pep"
os.chdir("/home/youngwookim_umass_edu/code/Chair")

In [3]:
from ptorch.splade_tree.c2_log import c2_log, reset_log_formatter
import hydra
import torch
from omegaconf import DictConfig
import os

from ptorch.splade_tree.datasets.dataloaders import DataLoaderWrapper
from ptorch.splade_tree.tasks.cross_encoder_trainer import CrossEncoderTransformerTrainer
from ptorch.splade_tree.tasks.pep_trainer import PEPTrainer
from ptorch.splade_tree.train_helper import get_train_dataset, get_val_evaluator, \
    get_train_loader, get_regularizer, build_val_loss_loader, load_model_resume, init_bert_bert_optim
from ptorch.splade_tree.datasets.datasets import PreLoadedDataset
from ptorch.splade_tree.models.models_utils import get_model
from ptorch.splade_tree.tasks.transformer_trainer import SiameseTransformerTrainer, TransformerTrainer
from ptorch.splade_tree.utils.utils import get_initialize_config, get_loss, set_seed_from_config
from cpath import yconfig_dir_path
from ptorch.splade_tree.utils.utils import get_initialize_config
from misc_lib import path_join
from ptorch.splade_tree.train import get_train_cls


INFO	C2 	03-02 04:10:26 C2 logging init
INFO	C2 	03-02 04:10:43 train main


In [6]:
from hydra import compose, initialize
from omegaconf import OmegaConf
conf_root = os.path.basename(yconfig_dir_path)
config_path = "../../../confs/hconfig"
print(config_path)
with initialize(version_base=None, config_path=config_path):
    cfg = compose(config_name="pep")
    print(OmegaConf.to_yaml(cfg))


../../../confs/hconfig
config:
  lr: 2.0e-05
  seed: 123
  gradient_accumulation_steps: 1
  weight_decay: 0.01
  pretrained_no_yamlconfig: false
  nb_iterations: 150000
  train_batch_size: 16
  eval_batch_size: 128
  record_frequency: 10000
  train_monitoring_freq: 5
  warmup_steps: 6000
  max_length: 256
  fp16: false
  matching_type: pep
  monitoring_ckpt: loss
  tokenizer_type: bert-base-uncased
  checkpoint_dir: /home/youngwookim_umass_edu/code/Chair/output/model/runs2/default_pep
  out_dir: ???
  loss: DistilMarginMSE
data:
  type: hard_negatives
  TRAIN:
    DATASET_PATH: data/msmarco/hard_negatives_scores/cross-encoder-ms-marco-MiniLM-L-6-v2-scores.pkl.gz
    D_COLLECTION_PATH: data/msmarco/full_collection
    Q_COLLECTION_PATH: data/msmarco/train_queries/queries
    QREL_PATH: data/msmarco/train_queries/qrels.json
  flops_queries: data/msmarco/all_dev_queries/
init_dict:
  model_type_or_dir: bert-base-uncased
  fp16: false
  max_length: 256



In [7]:
from ptorch.splade_tree.train import CONFIG_PATH, CONFIG_NAME

exp_dict, config, init_dict, _ = get_initialize_config(cfg, train=True)



config:
  lr: 2.0e-05
  seed: 123
  gradient_accumulation_steps: 1
  weight_decay: 0.01
  pretrained_no_yamlconfig: false
  nb_iterations: 150000
  train_batch_size: 16
  eval_batch_size: 128
  record_frequency: 10000
  train_monitoring_freq: 5
  warmup_steps: 6000
  max_length: 256
  fp16: false
  matching_type: pep
  monitoring_ckpt: loss
  tokenizer_type: bert-base-uncased
  checkpoint_dir: /home/youngwookim_umass_edu/code/Chair/output/model/runs2/default_pep
  out_dir: ???
  loss: DistilMarginMSE
data:
  type: hard_negatives
  TRAIN:
    DATASET_PATH: data/msmarco/hard_negatives_scores/cross-encoder-ms-marco-MiniLM-L-6-v2-scores.pkl.gz
    D_COLLECTION_PATH: data/msmarco/full_collection
    Q_COLLECTION_PATH: data/msmarco/train_queries/queries
    QREL_PATH: data/msmarco/train_queries/qrels.json
  flops_queries: data/msmarco/all_dev_queries/
init_dict:
  model_type_or_dir: bert-base-uncased
  fp16: false
  max_length: 256



In [8]:
from ptorch.splade_tree.models.models_utils import get_model, print_a
model = get_model(config, init_dict)
random_seed = set_seed_from_config(config)
optimizer, scheduler = init_bert_bert_optim(config, model)
iterations, regularizer = load_model_resume(config, model, optimizer, scheduler, random_seed)

if torch.cuda.device_count() > 1:
    c2_log.info(" --- use {} GPUs --- ".format(torch.cuda.device_count()))
    model = torch.nn.DataParallel(model)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


INFO	C2 	03-02 04:11:28  --- use 4 GPUs --- 


/home/youngwookim_umass_edu/work/miniconda3/envs/tf29/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


DataParallel(
  (module): TransformerWeightedSum(
    (bert_like): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bi

In [11]:

    data_train, train_mode = get_train_dataset(exp_dict)
    data_train: PreLoadedDataset = data_train

INFO	C2 	03-02 04:12:15 Preloading dataset from data/msmarco/full_collection


0it [00:00, ?it/s]

INFO	C2 	03-02 04:12:55 Preloading dataset from data/msmarco/train_queries/queries


0it [00:00, ?it/s]

INFO	C2 	03-02 04:13:58 QUERY SIZE = 502939


In [12]:
    loss = get_loss(config)
    regularizer = get_regularizer(config, model, regularizer)

    # fix for current in batch neg losses that break on last batch
    if config["loss"] in ("InBatchNegHingeLoss", "InBatchPairwiseNLL"):
        drop_last = True
    else:
        drop_last = False

    data_train, val_loss_loader = build_val_loss_loader(config, data_train, drop_last, train_mode, exp_dict)
    train_loader: DataLoaderWrapper = get_train_loader(config, data_train, drop_last, train_mode)
    val_evaluator = None

In [13]:
torch.cuda.memory_allocated()

439076864

In [ ]:
# #################################################################
# # TRAIN
# #################################################################
c2_log.info("+++++ BEGIN TRAINING +++++")
trainer_cls = get_train_cls(config.matching_type)
trainer: TransformerTrainer = trainer_cls(
    model=model,
    iterations=iterations, loss=loss, optimizer=optimizer,
    config=config, scheduler=scheduler,
    train_loader=train_loader, validation_loss_loader=val_loss_loader,
    validation_evaluator=val_evaluator,
    regularizer=regularizer)
trainer.train()


INFO	C2 	03-02 04:14:01 +++++ BEGIN TRAINING +++++
initialize trainer...
 --- total number parameters: 109483778
 === trainer config === 
 ========================= {'lr': 2e-05, 'seed': 123, 'gradient_accumulation_steps': 1, 'weight_decay': 0.01, 'pretrained_no_yamlconfig': False, 'nb_iterations': 150000, 'train_batch_size': 16, 'eval_batch_size': 128, 'record_frequency': 10000, 'train_monitoring_freq': 5, 'warmup_steps': 6000, 'max_length': 256, 'fp16': False, 'matching_type': 'pep', 'monitoring_ckpt': 'loss', 'tokenizer_type': 'bert-base-uncased', 'checkpoint_dir': '/home/youngwookim_umass_edu/code/Chair/output/model/runs2/default_pep', 'out_dir': '???', 'loss': 'DistilMarginMSE'}
Using FP16: False
INFO	C2 	03-02 04:14:03 Step 1
score torch.Size([4, 1])
score torch.Size([4, 1])
score torch.Size([4, 1])
score torch.Size([4, 1])
score torch.Size([4, 1])
score torch.Size([4, 1])
score torch.Size([4, 1])
score torch.Size([4, 1])
score torch.Size([4, 1])
score torch.Size([4, 1])
score to